## Widget Viewer

In [117]:
from PIL import Image
import piexif  
import json
import os
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

#Alows for floating windows:
%matplotlib qt


# Assuming 'sd_samplers' and 'IGNORED_INFO_KEYS' are defined elsewhere in your project
#  import them as needed

In [118]:
# inputs
expCopyPath = r"C:\pydev\forge\stable-diffusion-webui-forge\outputs\txt2img-images\2024-09-10" # RAW WINDOWS path to the experiment folder
in_percent_zoom = 50
in_y_center_percent = 30  # You can change this value as needed

In [119]:

# Global variable to track if the figure is open
fig = None
ax = None
image_plot = None


def display_or_update_image(image_path):
    global fig, ax1, ax2, image_plot1, image_plot2

    img = plt.imread(image_path)
    height, width, _ = img.shape

    percent_zoom = in_percent_zoom
    y_center_percent = in_y_center_percent  # You can change this value as needed

    # Calculate center and zoom region for the left image
    center_x = width // 2
    center_y = int(height * (y_center_percent / 100))  # Calculate y center based on percentage
    zoom_width = int(width * (percent_zoom / 100))
    zoom_height = int(height * (percent_zoom / 100))
    x_min, x_max = center_x - zoom_width // 2, center_x + zoom_width // 2
    y_min, y_max = center_y - zoom_height // 2, center_y + zoom_height // 2
    
    
    if fig is None:
        plt.ion()
        fig, (ax1, ax2) = plt.subplots(1, 2)  # Create two subplots

        # Left image (zoomed)
        image_plot1 = ax1.imshow(img)
        ax1.axis('off')
        ax1.set_xlim(x_min, x_max)
        ax1.set_ylim(y_max, y_min)

        # Right image (original)
        image_plot2 = ax2.imshow(img)
        ax2.axis('off')

        fig.canvas.toolbar.setVisible(False)
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0.1)  # Add some space between subplots
        plt.show()
    else:
        image_plot1.set_data(img)
        image_plot2.set_data(img)
        fig.canvas.draw_idle()

In [120]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image

image_dir = expCopyPath.replace("\\", "/")
checked_dir = 'checked'
deleted_dir = 'deleted'


widthsetting = '620px'

# Create subdirectories if they don't exist
for dir_name in [checked_dir, deleted_dir]:
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

# Get list of images in the directory
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
current_image_index = 0
imageCount = len(image_files)



# Create widgets
image_widget = widgets.Image(layout=widgets.Layout(width=widthsetting))  # Adjust width as needed
checked_button = widgets.Button(description='Checked')
deleted_button = widgets.Button(description='Deleted')

def read_info_from_image(image: Image.Image) -> tuple[str | None, dict]:
    items = (image.info or {}).copy()

    geninfo = items.pop('parameters', None)

    return geninfo, items

# Function to update the displayed image
def update_image():
    if 0 <= current_image_index < len(image_files):
        image_path = os.path.join(image_dir, image_files[current_image_index])
        image = Image.open(image_path)
        image_widget.value = image._repr_png_()  # Convert image to bytes for display
        geninfo, items = read_info_from_image(image)

        display_or_update_image(image_path)

        # # Method 1: Using Matplotlib (Recommended)
        # img = plt.imread(image_path)
        
        # height, width, _ = img.shape  # Get the height and width of the image

        # center_x = width // 2
        # center_y = height // 2

        # # Calculate the desired display range around the center
        # display_width = 400  # Adjust as needed
        # display_height = 400  # Adjust as needed

        # x_min = center_x - display_width // 2
        # x_max = center_x + display_width // 2
        # y_min = center_y - display_height // 2
        # y_max = center_y + display_height // 2

        # plt.figure(figsize=(16, 10)) 

        # plt.imshow(img)
        # plt.axis('off')

        # # Set the axes limits to show the center portion of the image
        # plt.xlim(x_min, x_max)
        # plt.ylim(y_max, y_min)  # Note: y_max comes first to avoid flipping

        # plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

        # plt.show()


    else:
        image_widget.value = b''  # Clear image if no more images

# Function to handle button clicks
def on_button_click(button):
    global current_image_index

    if 0 <= current_image_index < len(image_files):
        image_name = image_files[current_image_index]
        source_path = os.path.join(image_dir, image_name)

        if button == checked_button:
            target_dir = checked_dir
        else:  # button == deleted_button
            target_dir = deleted_dir

        target_path = os.path.join(target_dir, image_name)
        os.rename(source_path, target_path)

        del image_files[current_image_index]  # Remove from list
        if current_image_index >= len(image_files):
            current_image_index -= 1  # Move back if at the end

        update_image()
        update_text_block()

# Attach button click handlers
checked_button.on_click(on_button_click)
deleted_button.on_click(on_button_click)

# Create the HTML text block widget
text_block = widgets.HTML(
    value="""
    <div style="width: 400px; padding: 10px; border: 1px solid #ccc; margin-right: 10px;">
        <h3>Image Details</h3>
        <p>Filename: <span id="filename"></span></p>
        <p>Dimensions: <span id="dimensions"></span></p>
        <p>Remaining Files: {len(image_files)}</p>
    </div>
    """,
    layout=widgets.Layout(height='auto')  # Adjust height as needed
)

# Function to update the text block content
def update_text_block():
    if 0 <= current_image_index < len(image_files):
        image_path = os.path.join(image_dir, image_files[current_image_index])
        image = Image.open(image_path)
        geninfo, items = read_info_from_image(image)

        # Escape < and > in the geninfo string and add color styling
        escaped_geninfo = str(geninfo).replace("<", "&lt;").replace(">", "&gt;")


        negative_prompt_start = escaped_geninfo.find('Negative prompt:')
        if negative_prompt_start != -1:
            escaped_geninfo = (
                escaped_geninfo[:negative_prompt_start] +
                '<span style="color: red;">' + 
                escaped_geninfo[negative_prompt_start:] +
                '</span>'
            )
        
        # Color everything after "Steps:" in blue
        steps_start = escaped_geninfo.find('Steps:')
        if steps_start != -1:
            escaped_geninfo = (
                escaped_geninfo[:steps_start + len('Steps:')] +  # Include "Steps:" in the original color
                '<span style="color: blue;">' + 
                escaped_geninfo[steps_start + len('Steps:') :] +
                '</span>'
            )

        # Alternate colors before "Negative prompt:"
        # negative_prompt_start = escaped_geninfo.find('Negative prompt:')
        negative_prompt_start = escaped_geninfo.find('Steps:')
        if negative_prompt_start != -1:
            before_negative_prompt = escaped_geninfo[:negative_prompt_start]
            segments = before_negative_prompt.split(',')
            colored_segments = []
            for i, segment in enumerate(segments):
                color = "blue" if i % 2 == 0 else "green"
                colored_segments.append(f'<span style="color: {color};">{segment},</span>') 
            colored_before_negative_prompt = ''.join(colored_segments)
            escaped_geninfo = colored_before_negative_prompt + escaped_geninfo[negative_prompt_start:]

        text_block.value = f"""
        <div style="width: 400px; padding: 10px; border: 1px solid #ccc; margin-right: 10px;">
            <h3>Image Details</h3>
            <p>Filename: {image_files[current_image_index]}</p>
            <p>Dimensions: {image.width} x {image.height}</p>
            <p>Remaining Files: {len(image_files)}</p>
            <p>General Info: {escaped_geninfo}</p> 
        </div>
        """

    else:
        text_block.value = ""  # Clear text if no more images

# Update text block initially and when image changes
update_text_block()
update_image()  # Call after update_text_block to ensure both are updated

# Display widgets with the text block on the left
display(widgets.HBox([text_block, widgets.VBox([image_widget, widgets.HBox([checked_button, deleted_button])])]))





# # Initial display
# update_image()

# # Display widgets
# display(image_widget)
# display(widgets.HBox([checked_button, deleted_button]))